In [ ]:
using MAT                # for loading .mat files
using Statistics         # for mean, std, rms
using LinearAlgebra      # for vector operations
using Random             # for random number generation
using Plots              # for plotting

# === Load fixed Klauder wavelet ===
matfile = matopen("Trace_Klauder.mat")
Trace = read(matfile, "Trace")
close(matfile)

Trace_clean = Trace[(582-100):(582+100)]   # 201 samples
nt = length(Trace_clean)
sigRMS = sqrt(mean(Trace_clean .^2)) # RMS amplitude of clean signal to achieve the desired SNR level in the simulation 


# === Setup ===
M = 100                       # number of traces per realization
nRepeats = 100                # number of realizations per SNR and is simply the number of independant trials you run at each SNR value 
snr_dB = 20:-1:-60            # true SNR values in dB
snr_lin = 10 .^ (snr_dB ./ 10)   # convert dB to linear ratio
estimated_SNR_gamma = zeros(length(snr_dB), nRepeats) # we will hold the SNR estimates in linear scale for each SNR and each repeat

# === Loop over SNR levels and realizations ===
for (i, SNR_TRUE) in enumerate(snr_lin)
    for r in 1:nRepeats
        Traces = zeros(nt, M) # allocate matrix for M traces of length nt each colin is one trace

        # Generate traces with fixed signal + independent white noise
        for j in 1:M 
            noise = randn(nt)
            noise .*= (sigRMS / sqrt(SNR_TRUE) / sqrt(mean(noise .^2))) # Scale Noise to achieve requested SNR: P_noise  = P_signal / SNR_true --> RMS_noise = sig_rms / sqrt(snr_true)
            Traces[:, j] .= Trace_clean .+ noise
        end

        # Compute gamma (Equation 6) :  average pairwise normalized correlation 
        gamma_sum = 0.0
        for k in 1:M # Loop over first frame index (k)
            for l in (k+1):M # loop over second trace index (l), ensuring l > k to avoid duplicate pairs
                Rkl = sum(Traces[:, k] .* Traces[:, l]) # Step One: cross-correlation numerator between trace k and trace l --->  This computes the dot for dot between trace k and trace l 
                # physically: measure similarity between two signals 

                
                Rkk = sum(abs2, Traces[:, k]) # Energy of trace k
                Rll = sum(abs2, Traces[:, l]) # Energy of trace l
                gamma_sum += Rkl / sqrt(Rkk * Rll) # normallize corralation for each pair
            end
        end
        gamma = (2 / (M * (M - 1))) * gamma_sum # Average over all unit traces 
        gamma = clamp(gamma, 1e-10, 1 - 1e-10)  # numerical safety

        estimated_SNR_gamma[i, r] = gamma / (1 - gamma)
    end
end

# === Linear stats ===
mean_lin = mean(estimated_SNR_gamma, dims=2)
std_lin = std(estimated_SNR_gamma, dims=2)

# === Convert to dB ===
estimated_dB_mean = 10 .* log10.(mean_lin)
estimated_dB_upper = 10 .* log10.(mean_lin .+ std_lin)
estimated_dB_lower = 10 .* log10.(max.(mean_lin .- std_lin, 1e-10))  # avoid log of zero

# === Asymmetric errors ===
yNeg = estimated_dB_mean .- estimated_dB_lower
yPos = estimated_dB_upper .- estimated_dB_mean

# === PLOT: Estimated vs True SNR (with asymmetric error bars) ===
plot(snr_dB, estimated_dB_mean[:], 
     yerror = (yNeg[:], yPos[:]), 
     seriestype = :scatter, 
     marker = (:star5, 6), 
     line = (:solid, 1.5, :black),
     xlabel = "True SNR (dB)", 
     ylabel = "Estimated SNR (Eq. 6, dB)",
     title = "Figure 2a: γ-Based SNR Estimation (Averaged, Linear STD)",
     legend = false, 
     grid = true, 
     xlims = (-60, 20), 
     ylims = (-60, 20),
     framestyle = :box)

plot!(snr_dB, snr_dB, linestyle = :dash, linewidth = 2, color = :red)  # 1:1 reference line
